## Import

In [ ]:
import os
import sys
# Arbeisvezeichnis setzen
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())
print(f"Aktuelles Arbeitsverzeichnis: {os.getcwd()}")

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from utils.dataloader import HockeyDataset
from utils.transforms import train_transform, val_transform
from utils.model import HockeyActionModelResNet34

from sklearn.metrics import multilabel_confusion_matrix, ConfusionMatrixDisplay, classification_report, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import random


Aktuelles Arbeitsverzeichnis: c:\Users\hp\OneDrive\Desktop\DBU\wai81-ai-theory\ml_picture_recognition


Parameter


In [3]:
BATCH_SIZE = 8
NUM_WORKERS = 4
NUM_CLASSES = 4              # 4 Labels: Check, Neutral, Schuss, Tor
HIDDEN_DIM = 256
LEARNING_RATE = 1e-4
EPOCHS = 30
frames_per_clip = 100

In [4]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


## Dataset und Dataloader

In [5]:
# --- Dataset & DataLoader ---
train_dataset = HockeyDataset('data/labels_train.csv', 'data/train_frames', transform=train_transform, frames_per_clip=frames_per_clip)
val_dataset   = HockeyDataset('data/labels_val.csv', 'data/train_frames', transform=val_transform, frames_per_clip=frames_per_clip)


train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)


## Model

In [ ]:
# Modell
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = HockeyActionModelResNet34(embed_size=256, hidden_size=256, num_classes=4, num_layers=1)
model = model.to(device)
# Loss & Optimizer
criterion = torch.nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
c:\Users\hp\anaconda3\envs\picture\lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


In [ ]:
# Vorbereiten
best_val_f1 = 0                          # Beste bisher erreichte Validierungs-F1
best_epoch = 0                           # Merkt sich, in welcher Epoche das beste Modell war
early_stopping_counter = 0              # Zähler für Abbruchbedingung
patience = 5                             # Warten, wie viele Epochen ohne Verbesserung

train_losses = []                       # Verlauf Trainingsloss pro Epoche
train_f1_scores = []                    # Verlauf Trainings-F1 pro Epoche
val_losses = []                         # Verlauf Validierungs-Loss pro Epoche
val_f1_scores = []                      # Verlauf Validierungs-F1 pro Epoche

# Training vorbereiten
for epoch in range(EPOCHS):
    model.train()                       # Modell in Trainingsmodus setzen (aktiviert z. B. Dropout)
    running_loss = 0.0
    all_preds, all_targets = [], []    # Für spätere F1-Berechnung (alle Batches sammeln)

    for frames, labels in train_loader:
        frames = frames.to(device)                      # Eingabedaten auf GPU/CPU verschieben
        labels = labels.float().to(device)              # Labels ebenfalls auf Gerät (als Float wegen BCE)

        outputs = model(frames)                         # Vorwärtspass durch das Modell → Logits [B, 4]
        loss = criterion(outputs, labels)               # Multi-Label Loss (z. B. BCEWithLogitsLoss)

        optimizer.zero_grad()                           # Gradienten nullsetzen
        loss.backward()                                 # Backpropagation – Gradienten berechnen
        optimizer.step()                                # Parameter aktualisieren

        running_loss += loss.item()                     # Loss aufsummieren für Durchschnitt
        preds = (torch.sigmoid(outputs) > 0.5).float()  # Logits in Wahrscheinlichkeiten + Schwelle zu 0/1
        all_preds.append(preds.detach().cpu())          # detach() → keine Gradienten, auf CPU verschieben
        all_targets.append(labels.detach().cpu())       # Targets ebenfalls zur CPU

    # Durchschnittlicher Trainingsloss
    train_loss = running_loss / len(train_loader)

    # F1-Score über gesamten Trainingssatz (macro = Durchschnitt über Klassen)
    train_f1 = f1_score(torch.cat(all_targets), torch.cat(all_preds), average="macro", zero_division=0)

    # Logging für spätere Analyse oder Plotting
    train_losses.append(train_loss)
    train_f1_scores.append(train_f1)

    # Validierung
    model.eval()                          # Deaktiviert z. B. Dropout
    val_loss = 0.0
    val_preds, val_targets = [], []

    with torch.no_grad():                # Kein Gradienten-Tracking notwendig (spart Speicher)
        for frames, labels in val_loader:
            frames = frames.to(device)
            labels = labels.float().to(device)

            outputs = model(frames)                          # Forward-Pass
            loss = criterion(outputs, labels)                # Loss auf Validation Set berechnen

            val_loss += loss.item()
            preds = (torch.sigmoid(outputs) > 0.5).float()   # Schwelle: 0.5 → binäre Klassenentscheidung
            val_preds.append(preds.detach().cpu())
            val_targets.append(labels.detach().cpu())

    val_loss /= len(val_loader)  # Durchschnittlicher Validation Loss
    val_f1 = f1_score(torch.cat(val_targets), torch.cat(val_preds), average="macro", zero_division=0)

    val_losses.append(val_loss)
    val_f1_scores.append(val_f1)

    # Fortschritt ausgeben
    print(f"Epoch {epoch+1}: Train Loss {train_loss:.4f}, F1 {train_f1:.2f} | Val Loss {val_loss:.4f}, F1 {val_f1:.2f}")

    # Early Stopping
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1                         # Neuen Bestwert merken
        best_epoch = epoch + 1                       # Beste Epoche speichern
        early_stopping_counter = 0                   # Zähler zurücksetzen
        torch.save(model.state_dict(), 'models/best_resnet34_lstm.pth')  # Modell speichern
    else:
        early_stopping_counter += 1
        print(f"Early Stopping Counter: {early_stopping_counter}/{patience}")
        if early_stopping_counter >= patience:
            print(f"⏹️ Early stopping triggered at epoch {epoch+1}. Best epoch: {best_epoch}")
            break


Epoch 1: Train Loss 0.6412, F1 0.21 | Val Loss 0.5857, F1 0.17
Epoch 2: Train Loss 0.5042, F1 0.36 | Val Loss 0.4887, F1 0.38
Epoch 3: Train Loss 0.3820, F1 0.58 | Val Loss 0.4771, F1 0.49
Epoch 4: Train Loss 0.3058, F1 0.83 | Val Loss 0.4516, F1 0.60
Epoch 5: Train Loss 0.2456, F1 0.88 | Val Loss 0.5203, F1 0.54
Early Stopping Counter: 1/5
Epoch 6: Train Loss 0.1886, F1 0.93 | Val Loss 0.3571, F1 0.73
Epoch 7: Train Loss 0.1584, F1 0.93 | Val Loss 0.2925, F1 0.80
Epoch 8: Train Loss 0.1339, F1 0.94 | Val Loss 0.3694, F1 0.72
Early Stopping Counter: 1/5
Epoch 9: Train Loss 0.0893, F1 0.97 | Val Loss 0.4230, F1 0.70
Early Stopping Counter: 2/5
Epoch 10: Train Loss 0.0785, F1 0.98 | Val Loss 0.4805, F1 0.68
Early Stopping Counter: 3/5
Epoch 11: Train Loss 0.0975, F1 0.94 | Val Loss 0.3567, F1 0.76
Early Stopping Counter: 4/5
Epoch 12: Train Loss 0.0931, F1 0.95 | Val Loss 0.6446, F1 0.68
Early Stopping Counter: 5/5
⏹️ Early stopping triggered at epoch 12. Best epoch: 7


In [ ]:
# Klassen-Labels
class_names = ['Check', 'Neutral', 'Schuss', 'Tor']

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for frames, labels in val_loader:
        frames = frames.to(device)
        labels = labels.to(device)

        outputs = model(frames)
        predicted = (torch.sigmoid(outputs) > 0.5).int()

        all_preds.append(predicted.cpu())
        all_labels.append(labels.cpu().int())

# Tensors stapeln
all_preds = torch.cat(all_preds).numpy()  # Shape [N, 4]
all_labels = torch.cat(all_labels).numpy()

# Multilabel Confusion Matrix (eine 2x2-Matrix pro Klasse)
mcm = multilabel_confusion_matrix(all_labels, all_preds)

# Plot: jede Klasse einzeln anzeigen
fig, axes = plt.subplots(2, 2, figsize=(10, 10))
axes = axes.flatten()

for i in range(len(class_names)):
    ConfusionMatrixDisplay(
        confusion_matrix=mcm[i],
        display_labels=[f'not {class_names[i]}', class_names[i]]
    ).plot(ax=axes[i], cmap='Blues', values_format='d')
    axes[i].set_title(f"Klasse: {class_names[i]}")

plt.tight_layout()
plt.show()


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [8]:
# --- Save Model ---
torch.save(model.state_dict(), 'models/best_resnet34_lstm.pth')